In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as keras
import tf2lib as tl
import tf2gan as gan
import DLlib as dl
import pylib as py
import wflib as wf
import data

import itertools

In [2]:
dataset_dir = '../../OneDrive - Universidad Católica de Chile/Documents/datasets/'
dataset_hdf5_1 = 'JGalgani_GC_192_complex_2D.hdf5'
acqs, out_maps = data.load_hdf5(dataset_dir,dataset_hdf5_1, 12, end=20, MEBCRN=True)

In [3]:
len_dataset,ne,hgt,wdt,n_ch = np.shape(acqs)
_,n_out,_,_,_ = np.shape(out_maps)

In [4]:
metric_model = dl.perceptual_metric(input_shape=(1,hgt,wdt,n_ch))
cosine_loss = tf.losses.CosineSimilarity()
mse_loss = tf.losses.MeanSquaredError()
mae_loss = tf.losses.MeanAbsoluteError()
mmd_loss = dl.MMD()

In [5]:
A = acqs[12:13]
A_abs = tf.math.sqrt(tf.reduce_sum(tf.square(A),axis=-1,keepdims=True))
idx_pairs = list(itertools.combinations(range(ne), 2))
# np.random.shuffle(idx_pairs)
for idx_a, idx_b in idx_pairs:
    A_a = metric_model(A[:,idx_a:idx_a+1,:,:,:], training=False)
    A_b = metric_model(A[:,idx_b:idx_b+1,:,:,:], training=False)
    loss = cosine_loss(A_a[0], A_b[0])/len(A_a)
    for l in range(1,len(A_a)):
        loss += cosine_loss(A_a[l], A_b[l])/len(A_a)
    print('Cosine loss of pair', idx_a, idx_b, '-->', loss.numpy())

Cosine loss of pair 0 1 --> -0.9130012
Cosine loss of pair 0 2 --> -0.9223494
Cosine loss of pair 0 3 --> -0.8985305
Cosine loss of pair 0 4 --> -0.8720776
Cosine loss of pair 0 5 --> -0.874866
Cosine loss of pair 1 2 --> -0.90359807
Cosine loss of pair 1 3 --> -0.9285376
Cosine loss of pair 1 4 --> -0.8954591
Cosine loss of pair 1 5 --> -0.8946016
Cosine loss of pair 2 3 --> -0.8927279
Cosine loss of pair 2 4 --> -0.926437
Cosine loss of pair 2 5 --> -0.8910793
Cosine loss of pair 3 4 --> -0.9239044
Cosine loss of pair 3 5 --> -0.9587109
Cosine loss of pair 4 5 --> -0.92042714


In [6]:
for idx_a, idx_b in idx_pairs:
    loss = mse_loss(A_abs[:,idx_a:idx_a+1,:,:,:], A_abs[:,idx_b:idx_b+1,:,:,:])
    print('MSE loss of pair', idx_a, idx_b, '-->', loss.numpy())

MSE loss of pair 0 1 --> 0.004197146277874708
MSE loss of pair 0 2 --> 0.006818694528192282
MSE loss of pair 0 3 --> 0.02551659569144249
MSE loss of pair 0 4 --> 0.028088195249438286
MSE loss of pair 0 5 --> 0.03331875056028366
MSE loss of pair 1 2 --> 0.0033280616626143456
MSE loss of pair 1 3 --> 0.012029075063765049
MSE loss of pair 1 4 --> 0.015812862664461136
MSE loss of pair 1 5 --> 0.017871081829071045
MSE loss of pair 2 3 --> 0.00979998242110014
MSE loss of pair 2 4 --> 0.008977223187685013
MSE loss of pair 2 5 --> 0.014361000619828701
MSE loss of pair 3 4 --> 0.0034015143755823374
MSE loss of pair 3 5 --> 0.001998116262257099
MSE loss of pair 4 5 --> 0.004105175379663706


In [7]:
for idx_a, idx_b in idx_pairs:
    loss = mae_loss(A_abs[:,idx_a:idx_a+1,:,:,:], A_abs[:,idx_b:idx_b+1,:,:,:])
    print('MAE loss of pair', idx_a, idx_b, '-->', loss.numpy())

MAE loss of pair 0 1 --> 0.034382838755846024
MAE loss of pair 0 2 --> 0.04408960044384003
MAE loss of pair 0 3 --> 0.0939849242568016
MAE loss of pair 0 4 --> 0.09534742683172226
MAE loss of pair 0 5 --> 0.11112691462039948
MAE loss of pair 1 2 --> 0.030117562040686607
MAE loss of pair 1 3 --> 0.06340975314378738
MAE loss of pair 1 4 --> 0.06700155138969421
MAE loss of pair 1 5 --> 0.08028073608875275
MAE loss of pair 2 3 --> 0.05350258946418762
MAE loss of pair 2 4 --> 0.054196227341890335
MAE loss of pair 2 5 --> 0.0699073076248169
MAE loss of pair 3 4 --> 0.029076602309942245
MAE loss of pair 3 5 --> 0.0232064351439476
MAE loss of pair 4 5 --> 0.03253031149506569


In [8]:
for idx_a, idx_b in idx_pairs:
    mmd_loss.reset_state()
    loss = mmd_loss(A_abs[:,idx_a:idx_a+1,:,:,:], A_abs[:,idx_b:idx_b+1,:,:,:])
    print('MMD loss of pair', idx_a, idx_b, '-->', loss.numpy())

MMD loss of pair 0 1 --> 0.0041971505
MMD loss of pair 0 2 --> 0.006818682
MMD loss of pair 0 3 --> 0.0255166
MMD loss of pair 0 4 --> 0.028088182
MMD loss of pair 0 5 --> 0.033318758
MMD loss of pair 1 2 --> 0.0033280551
MMD loss of pair 1 3 --> 0.012029067
MMD loss of pair 1 4 --> 0.015812859
MMD loss of pair 1 5 --> 0.017871082
MMD loss of pair 2 3 --> 0.009799972
MMD loss of pair 2 4 --> 0.008977219
MMD loss of pair 2 5 --> 0.014360987
MMD loss of pair 3 4 --> 0.0034015104
MMD loss of pair 3 5 --> 0.001998119
MMD loss of pair 4 5 --> 0.0041051805


In [13]:
loss /=10
loss.numpy()

0.00041051806